In [3]:
from datascience import *
import numpy as np
import re
from re import search
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [4]:
# scrape_table function from video demo
import pandas as pd
def scrape_table(table):
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    return Table.from_df(df)

In [5]:
r = requests.get("https://www.numbeo.com/cost-of-living/prices_by_country.jsp?itemId=6&regionCode=019&displayCurrency=CAD")
coke_text = BeautifulSoup(r.text, "html.parser")
coke_text_table = coke_text.find_all("table")[1]
coke_table = scrape_table(coke_text_table).drop("Rank")
coke_table

Country,Coke/Pepsi (0.33 liter bottle)
Kazakhstan,0.78
Paraguay,1.18
Portugal,1.96
Greece,2.53
Latvia,1.56
Iran,1.16
Morocco,0.63
Panama,2.04
Guatemala,1.38
Iraq,0.58


In [6]:
def convert(cur_from, cur_to, amount):
    payload = {
        'from': cur_from,
        'to': cur_to,
        'amount': amount,
        'api_key': '0a5659758c74b6dd01f96fdbc8fab87d'
    }
    r = requests.get('https://api.currencyscoop.com/v1/convert', params = payload)
    currency = r.json()
    return currency['response']['value']

In [7]:
#stolen shamlessly from example code
currency_url = "https://currencyscoop.com/supported-currencies"
currency_request = requests.get(currency_url)
currency_soup = BeautifulSoup(currency_request.text, 'html.parser')
currency_table = currency_soup.find("table")
currency_codes = scrape_table(currency_table)

codes = make_array()
for each_country in currency_codes.column("Currency"):
    codes = np.append(codes, each_country[-3:])

currency_codes = currency_codes.with_column("Code", codes)

def strip_code(country):
    return country[:-3]

currency_codes["Currency"] = currency_codes.apply(strip_code, "Currency")

In [8]:
import pycountry

mapping = {country.name: country.alpha_2 for country in pycountry.countries}
coke_table.with_column("Country Code", coke_table.apply(lambda x: mapping.get(x), "Country"))

Country,Coke/Pepsi (0.33 liter bottle),Country Code
Kazakhstan,0.78,KZ
Paraguay,1.18,PY
Portugal,1.96,PT
Greece,2.53,GR
Latvia,1.56,LV
Iran,1.16,None
Morocco,0.63,MA
Panama,2.04,PA
Guatemala,1.38,GT
Iraq,0.58,IQ


In [11]:
r = requests.get("https://www.iban.com/currency-codes")
currency_code_table = scrape_table(BeautifulSoup(r.text, "html.parser").find_all("table")[0])
currency_code_table

Country,Currency,Code,Number
AFGHANISTAN,Afghani,AFN,971
ÅLAND ISLANDS,Euro,EUR,978
ALBANIA,Lek,ALL,8
ALGERIA,Algerian Dinar,DZD,12
AMERICAN SAMOA,US Dollar,USD,840
ANDORRA,Euro,EUR,978
ANGOLA,Kwanza,AOA,973
ANGUILLA,East Caribbean Dollar,XCD,951
ANTARCTICA,No universal currency,nan,nan
ANTIGUA AND BARBUDA,East Caribbean Dollar,XCD,951


In [16]:
def get_currency_code(country):
    i = np.where(currency_code_table.column("Country") == country.upper())
    return currency_code_table.column("Code")[i]

def clean_currency_code(code):
    cleaned_code = code[:-3]
    return cleaned_code

coke_table = coke_table.with_column("Currency Code", coke_table.apply(get_currency_code, "Country"))
coke_table = coke_table.with_column("Currency Code", coke_table.apply(clean_currency_code, "Currency Code"))

coke_table

Country,Coke/Pepsi (0.33 liter bottle),Currency Code
Kazakhstan,0.78,['KZT']
Paraguay,1.18,['PYG']
Portugal,1.96,['EUR']
Greece,2.53,['EUR']
Latvia,1.56,['EUR']
Iran,1.16,[]
Morocco,0.63,['MAD']
Panama,2.04,['PAB' 'USD']
Guatemala,1.38,['GTQ']
Iraq,0.58,['IQD']


In [37]:
# tidy up currency symbol and GST rate
currencies = make_array()
gst_rates = make_array()

# loop through each country
for each_country in coke_table.column("Country"):
    # grab currency symbol only
  

    # grab GST rate only -- some countries don't have one, so I used regex to find perfect match
    match = search("\d\d%", each_country)
    if match != None:
        gst_rates = np.append(gst_rates, match.group(0))

    else:
        gst_rates = np.append(gst_rates, '0%')

# tack on new columns
coke_table = coke_table.with_column("GST Rate", gst_rates)
coke_table

Country,Coke/Pepsi (0.33 liter bottle),Local Price,GST Rate
Thailand,0.78,0.78,0%
Libya,0.63,0.63,0%
Costa Rica,2.21,2.21,0%
Sweden,2.58,2.58,0%
Vietnam,0.68,0.68,0%
Poland,1.72,1.72,0%
Bulgaria,1.57,1.57,0%
Jordan,0.71,0.71,0%
Kuwait,0.73,0.73,0%
Nigeria,0.5,0.5,0%
